#Imports and Ajusting file


In [4]:
import cv2
import pandas as pd
import os
import shutil
import time
import math
import zipfile
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
import string
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pydicom
import gdown
# from google.colab.patches import cv2_imshow

In [5]:
def download(id): 
    url = 'https://drive.google.com/uc?id=' + str(id)
    gdown.download(url, output = None, quiet = False)

def unzip(path): #Função para unzip
    zip = zipfile.ZipFile(path)
    zip.extractall()
    zip.close()

In [6]:
# https://drive.google.com/file/d/1ev-r31j8oRzDlKM_toaeADO2psrA_XXm/view?usp=sharing

download('1ev-r31j8oRzDlKM_toaeADO2psrA_XXm')
#/mnt/c/Códigos_VS/X-rayBodyPart/archive.zip
unzip('/mnt/c/Códigos_VS/X-rayBodyPart/archive.zip')

Downloading...
From: https://drive.google.com/uc?id=1ev-r31j8oRzDlKM_toaeADO2psrA_XXm
To: /mnt/c/Códigos_VS/X-rayBodyPart/archive.zip
100%|██████████| 269M/269M [00:53<00:00, 5.05MB/s]


In [7]:
train_df = pd.read_csv('/mnt/c/Códigos_VS/X-rayBodyPart/train.csv')
test_df = pd.read_csv('/mnt/c/Códigos_VS/X-rayBodyPart/sample_submission.csv')

In [8]:
test_df.head(5)

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10001001190452685542...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,1.2.826.0.1.3680043.8.498.10022667601042710442...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2,1.2.826.0.1.3680043.8.498.10024395388921105474...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
3,1.2.826.0.1.3680043.8.498.10026689165626095651...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4,1.2.826.0.1.3680043.8.498.10035936364561920980...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...


In [9]:
train_df.head(5)

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3


#Pre-processing

In [10]:
print("Columns:")
print(train_df.columns, "\n")

print("Types in columns:")
print(train_df.dtypes, "\n")

print("Types in info:")
print(train_df.describe(), "\n")

Columns:
Index(['SOPInstanceUID', 'Target'], dtype='object') 

Types in columns:
SOPInstanceUID    object
Target            object
dtype: object 

Types in info:
                                           SOPInstanceUID Target
count                                                1738   1738
unique                                               1738     41
top     1.2.826.0.1.3680043.8.498.10025629581362719970...     3 
freq                                                    1    724 



##Fixing folders

In [11]:
class Image_processing():
  def __init__(self, path):
    self.path = "/mnt/c/Códigos_VS/X-rayBodyPart/" + path + "/"
    self.fileList = []

  def fixFolders(self, name):
    filelist = []
    for root, dirs, files in os.walk(self.path):
      # os.walk generate the file names in a directory tree by walking the tree either top-down or bottom-up
      for file in files:
            #append the file name to the list
        filelist.append(os.path.join(root,file))
    
    self.fileList = filelist
    os.mkdir("/mnt/c/Códigos_VS/X-rayBodyPart/" + name + "/")

    for filename in filelist:
        dicom = pydicom.dcmread(filename)
        img = dicom.pixel_array
        im = Image.fromarray(img.astype(np.uint8))
        var = filename.split("/")
        patient = var[5]
        code = var[6]
        im.save("/mnt/c/Códigos_VS/X-rayBodyPart/" + name + "/" + var[-1][:-6] + ".bmp")


In [12]:
# shutil.rmtree("/content/trainRepaired")
# shutil.rmtree("/content/trainRepaired2")
# shutil.rmtree("/content/testRepaired")

In [13]:
try:
  image_processing = Image_processing("train")
  image_processing.fixFolders("trainRepaired")

  image_processing2 = Image_processing("test")
  image_processing2.fixFolders("testRepaired")
except FileExistsError:
  # /mnt/c/Códigos_VS/X-rayBodyPart/train.csv
  shutil.rmtree("/mnt/c/Códigos_VS/X-rayBodyPart/" + "trainRepaired")
  shutil.rmtree("/mnt/c/Códigos_VS/X-rayBodyPart/" + "testRepaired" )

  image_processing = Image_processing("train")
  image_processing.fixFolders("trainRepaired")

  image_processing2 = Image_processing("test")
  image_processing2.fixFolders("testRepaired")

In [14]:
train_df

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3
...,...,...
1733,1.2.826.0.1.3680043.8.498.99711016467406172920...,14
1734,1.2.826.0.1.3680043.8.498.99727280485333900933...,11
1735,1.2.826.0.1.3680043.8.498.99935008698914755885...,3
1736,1.2.826.0.1.3680043.8.498.99969671777553755311...,2


In [15]:
lista=[[[1]],[2], [3]]

In [16]:
def dcmtag2table(folder, list_of_tags):

    list_of_tags = list_of_tags.copy()
    items = []
    table = []
    filelist = []

    for root, dirs, files in os.walk(folder, topdown=False):
#Adiciona os caminhos de cada imagem na lista
        for name in files:
            filelist.append(os.path.join(root, name))
    aux= 0
    for _f in filelist:
      #Em cada caminho da imagem(.dcm)
        try:
            ds = pydicom.dcmread(_f, stop_before_pixels=True)
            #Pega o Dataset de cada imagem e ler no próximo for
            items = []
            items.append(_f)
            #Adiciono o caminho em "items"
            for _tag in list_of_tags:
                if _tag in ds:
                  
                    items.append(ds.data_element(_tag).value)
                    #Adiciono a features "PhotometricInterpretation", "BitsAllocated" and "SOPInstanceUID" em "items"
                else:
                    items.append("Not found")
            
            table.append(items)
            #Adiciona tudo em uma outra lista
        except:
            print("Skipping non-DICOM: " + _f)

    df = pd.DataFrame(table, columns=["Filename", "PhotometricInterpretation", "BitsAllocated","SOPInstanceUID"])
    #Crio um data set com todos os caminhos Photometric, Bits and SOP
    return df

*   SOP Instance UID - Unique Identification
*   SOP Class UID    - 

In [28]:
tags = ['PhotometricInterpretation','BitsAllocated', 'SOPInstanceUID' ]
dicom_tags_train =  dcmtag2table('/mnt/c/Códigos_VS/X-rayBodyPart/train', tags)
dicom_tags_test = dcmtag2table('/mnt/c/Códigos_VS/X-rayBodyPart/test', tags)

In [29]:
dicom_tags_train

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID
0,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.65452424240994805812...
1,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.29960944389705651248...
2,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.57792439353439709695...
3,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.37634180084111795685...
4,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12781870272366327239...
...,...,...,...,...
1733,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.88201401328037140717...
1734,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.88978469234834814481...
1735,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.33303514480609398517...
1736,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.13310561881659972417...


In [30]:
print(dicom_tags_train.PhotometricInterpretation.value_counts())

MONOCHROME1    1659
MONOCHROME2      79
Name: PhotometricInterpretation, dtype: int64


In [31]:
train = dicom_tags_train.merge(train_df, on =  'SOPInstanceUID')
test = dicom_tags_test.merge(test_df,on =  'SOPInstanceUID')

In [32]:
train.head()

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target
0,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.65452424240994805812...,3
1,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.29960944389705651248...,3
2,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.57792439353439709695...,3
3,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.37634180084111795685...,3
4,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12781870272366327239...,3


In [33]:
target = list(train["Target"])

In [34]:
#Let's  see our Target distribution

bodyparts = {
0 : 'Abdomen' ,
1 :'Ankle' ,
2 :'Cervical Spine',
3 : 'Chest' ,
4 :'Clavicles' ,
5 :'Elbow' ,
6 :'Feet' ,
7 : 'Finger' ,
8 : 'Forearm' ,
9 : 'Hand' ,
10 : 'Hip' ,
11 : 'Knee' ,
12 : 'Lower Leg' ,
13 : 'Lumbar Spine' ,
14 : 'Others' ,
15 :'Pelvis',
16 :'Shoulder' ,
17 :'Sinus' ,
18 : 'Skull' ,
19 : 'Thigh' ,
20 :'Thoracic Spine',
21: 'Wrist',
}

In [35]:
labels = ['Abdomen', 'Ankle', 'Cervical Spine',
       'Chest', 'Clavicles', 'Elbow', 'Feet', 'Finger', 'Forearm', 'Hand',
       'Hip', 'Knee', 'Lower Leg', 'Lumbar Spine', 'Others', 'Pelvis',
       'Shoulder', 'Sinus', 'Skull', 'Thigh', 'Thoracic Spine', 'Wrist']

In [36]:
#Create Label from target
def no_to_label(label):
    label_list_string = []
    trimed_label = label.rstrip()
    label_list = trimed_label.split(" ")
    label_list = [int(i) for i in label_list]
    for label in label_list:
        label_list_string.append(bodyparts[label])
    label_string = ' and '.join(label_list_string)
    return label_string

target_list = train['Target'].tolist()
label_column = []

for label in tqdm(target_list):
    label_string = no_to_label(label)
    label_column.append(label_string)

train['Label'] = label_column
train['Label'].unique()

100%|██████████| 1738/1738 [00:00<00:00, 267276.54it/s]


array(['Chest', 'Hand and Wrist', 'Cervical Spine', 'Pelvis', 'Knee',
       'Wrist', 'Others', 'Finger', 'Hand', 'Feet', 'Elbow', 'Ankle',
       'Lumbar Spine and Thoracic Spine', 'Thoracic Spine',
       'Lumbar Spine', 'Abdomen', 'Skull', 'Hip', 'Shoulder',
       'Finger and Hand', 'Ankle and Knee and Lower Leg',
       'Abdomen and Chest', 'Sinus', 'Lower Leg', 'Chest and Finger',
       'Thigh', 'Ankle and Lower Leg', 'Hip and Knee and Thigh',
       'Clavicles', 'Elbow and Forearm', 'Forearm', 'Abdomen and Pelvis',
       'Ankle and Feet', 'Hip and Thigh', 'Pelvis and Thigh',
       'Knee and Lower Leg', 'Forearm and Shoulder', 'Chest and Pelvis',
       'Cervical Spine and Thoracic Spine',
       'Cervical Spine and Lumbar Spine and Thoracic Spine',
       'Chest and Feet'], dtype=object)

In [37]:
train

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target,Label
0,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.65452424240994805812...,3,Chest
1,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.29960944389705651248...,3,Chest
2,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.57792439353439709695...,3,Chest
3,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.37634180084111795685...,3,Chest
4,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12781870272366327239...,3,Chest
...,...,...,...,...,...,...
1733,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.88201401328037140717...,10,Hip
1734,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.88978469234834814481...,15,Pelvis
1735,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.33303514480609398517...,2,Cervical Spine
1736,/mnt/c/Códigos_VS/X-rayBodyPart/train/train/tr...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.13310561881659972417...,3,Chest


In [ ]:
train.Label.value_counts()

Delete every row with more than one organ

In [39]:
frase = "Ankle and Knee and Lower Leg"
if "and" in frase:
    print("Ele identificou")

Ele identificou


In [40]:
aux = list(train["Label"])
index=[]
for i in range(len(aux)):
    if "and" in aux[i]:
        index.append(i)

In [41]:
train.drop(index, axis=0, inplace=True)

In [42]:
train.Label.value_counts()


Chest             724
Others            120
Knee              102
Abdomen            80
Cervical Spine     77
Feet               70
Pelvis             67
Wrist              63
Ankle              41
Shoulder           40
Lumbar Spine       39
Elbow              23
Sinus              23
Lower Leg          19
Hip                19
Thoracic Spine     15
Finger             15
Forearm            12
Skull              10
Clavicles           9
Thigh               7
Name: Label, dtype: int64